# DVIDS Daily Digest - U.S. Navy News Aggregator

This notebook scrapes daily news, images, and videos from DVIDS (Defense Visual Information Distribution Service) and generates a modern web interface sorted by geography/location.

**Created by @ianellisjones and IEJ Media**

---

## Setup Instructions

1. Get a free DVIDS API key at: https://api.dvidshub.net/docs
2. Enter your API key in the configuration cell below
3. Run all cells to generate the daily digest

In [ ]:
# Install dependencies
!pip install requests -q

In [ ]:
# ============================================================================
# CONFIGURATION - Enter your DVIDS API key here
# ============================================================================

DVIDS_API_KEY = "YOUR_API_KEY_HERE"  # <-- Replace with your actual API key

# How many hours back to search (24 = last day)
LOOKBACK_HOURS = 24

# Military branches to include
BRANCHES = ["Navy", "Marines", "Coast Guard", "Joint"]

# Content types to fetch
CONTENT_TYPES = ["news", "image", "video"]

In [ ]:
# ============================================================================
# IMPORTS AND SETUP
# ============================================================================

import re
import json
import time
from datetime import datetime, timedelta
from typing import List, Dict, Optional
from dataclasses import dataclass, asdict, field
from collections import defaultdict
from pathlib import Path
import requests
from IPython.display import HTML, display, clear_output
from google.colab import files

# API Configuration
DVIDS_API_BASE = "https://api.dvidshub.net"
MAX_RESULTS_PER_QUERY = 100
USER_AGENT = 'DVIDS-News-Aggregator/1.0 (Python; Colab)'

print("Setup complete!")

In [ ]:
# ============================================================================
# DATA CLASSES AND MAPPINGS
# ============================================================================

@dataclass
class DVIDSItem:
    """Data class representing a DVIDS content item."""
    id: str
    title: str
    description: str
    type: str
    branch: str
    unit_name: str
    date_published: str
    timestamp: str
    country: str
    state: str
    city: str
    location_display: str
    url: str
    thumbnail_url: str
    keywords: List[str] = field(default_factory=list)
    credit: str = ""
    duration: str = ""
    aspect_ratio: str = ""


@dataclass
class DailyDigest:
    """Data class for a day's worth of DVIDS content."""
    date: str
    items: List[DVIDSItem]
    total_count: int
    by_country: Dict[str, int]
    by_type: Dict[str, int]
    by_branch: Dict[str, int]


# Country code to name mapping
COUNTRY_NAMES = {
    "US": "United States", "JP": "Japan", "KR": "South Korea",
    "PH": "Philippines", "AU": "Australia", "DE": "Germany",
    "IT": "Italy", "ES": "Spain", "UK": "United Kingdom",
    "GB": "United Kingdom", "BH": "Bahrain", "AE": "United Arab Emirates",
    "QA": "Qatar", "KW": "Kuwait", "DJ": "Djibouti",
    "GR": "Greece", "TR": "Turkey", "PL": "Poland",
    "NO": "Norway", "SE": "Sweden", "FI": "Finland",
    "IS": "Iceland", "CA": "Canada", "MX": "Mexico",
    "PR": "Puerto Rico", "GU": "Guam", "VI": "U.S. Virgin Islands",
    "CU": "Cuba (Guantanamo Bay)", "SG": "Singapore",
    "TH": "Thailand", "VN": "Vietnam", "IN": "India",
    "AF": "Afghanistan", "IQ": "Iraq", "SY": "Syria",
    "JO": "Jordan", "IL": "Israel", "EG": "Egypt",
    "SA": "Saudi Arabia", "OM": "Oman", "YE": "Yemen",
}

# Region groupings
REGION_MAP = {
    "United States": "CONUS",
    "Puerto Rico": "Caribbean",
    "U.S. Virgin Islands": "Caribbean",
    "Cuba (Guantanamo Bay)": "Caribbean",
    "Guam": "Indo-Pacific",
    "Japan": "Indo-Pacific",
    "South Korea": "Indo-Pacific",
    "Philippines": "Indo-Pacific",
    "Australia": "Indo-Pacific",
    "Singapore": "Indo-Pacific",
    "Thailand": "Indo-Pacific",
    "Vietnam": "Indo-Pacific",
    "India": "Indo-Pacific",
    "Germany": "Europe",
    "Italy": "Europe",
    "Spain": "Europe",
    "United Kingdom": "Europe",
    "Greece": "Europe",
    "Turkey": "Europe",
    "Poland": "Europe",
    "Norway": "Europe",
    "Sweden": "Europe",
    "Finland": "Europe",
    "Iceland": "Europe",
    "Bahrain": "Middle East",
    "United Arab Emirates": "Middle East",
    "Qatar": "Middle East",
    "Kuwait": "Middle East",
    "Saudi Arabia": "Middle East",
    "Oman": "Middle East",
    "Iraq": "Middle East",
    "Syria": "Middle East",
    "Jordan": "Middle East",
    "Israel": "Middle East",
    "Egypt": "Middle East",
    "Yemen": "Middle East",
    "Djibouti": "Africa",
    "Afghanistan": "Central Asia",
    "Canada": "North America",
    "Mexico": "Central America",
}

print("Data classes loaded!")

In [ ]:
# ============================================================================
# API FUNCTIONS
# ============================================================================

def make_api_request(endpoint: str, params: Dict, retries: int = 3) -> Optional[Dict]:
    """Make a request to the DVIDS API with retry logic."""
    url = f"{DVIDS_API_BASE}{endpoint}"
    params["api_key"] = DVIDS_API_KEY

    headers = {
        "User-Agent": USER_AGENT,
        "Accept": "application/json",
    }

    for attempt in range(retries):
        try:
            response = requests.get(url, params=params, headers=headers, timeout=30)

            if response.status_code == 200:
                return response.json()
            elif response.status_code == 403:
                print(f"ERROR: API key invalid. Get one at https://api.dvidshub.net/docs")
                return None
            elif response.status_code == 429:
                wait_time = 2 ** attempt
                print(f"Rate limited. Waiting {wait_time}s...")
                time.sleep(wait_time)
                continue
            else:
                print(f"HTTP {response.status_code}")

        except requests.RequestException as e:
            print(f"Request error (attempt {attempt + 1}): {e}")
            if attempt < retries - 1:
                time.sleep(2 ** attempt)

    return None


def search_dvids(content_type: str, branch: str = None,
                 from_date: str = None, to_date: str = None,
                 max_results: int = 50) -> List[Dict]:
    """Search DVIDS for content."""
    params = {
        "type": content_type,
        "max_results": min(max_results, MAX_RESULTS_PER_QUERY),
        "sort": "date",
    }

    if branch:
        params["branch"] = branch
    if from_date:
        params["from_date"] = from_date
    if to_date:
        params["to_date"] = to_date

    result = make_api_request("/search", params)

    if result and "results" in result:
        return result["results"]

    return []


print("API functions loaded!")

In [ ]:
# ============================================================================
# DATA PROCESSING
# ============================================================================

def parse_dvids_item(raw_item: Dict) -> Optional[DVIDSItem]:
    """Parse a raw DVIDS API result into a DVIDSItem."""
    try:
        item_id = str(raw_item.get("id", ""))
        title = raw_item.get("title", "Untitled")
        description = raw_item.get("description", raw_item.get("short_description", ""))
        content_type = raw_item.get("type", "unknown")

        # Clean description
        if description:
            description = re.sub(r'<[^>]+>', '', description)
            description = description[:500] + "..." if len(description) > 500 else description

        branch = raw_item.get("branch", "Unknown")
        unit_name = raw_item.get("unit_name", "Unknown Unit")

        # Date handling
        date_published = raw_item.get("date_published", raw_item.get("date", ""))
        if date_published:
            try:
                if "T" in date_published:
                    dt = datetime.fromisoformat(date_published.replace("Z", "+00:00"))
                else:
                    dt = datetime.strptime(date_published[:10], "%Y-%m-%d")
                timestamp = dt.isoformat()
                date_published = dt.strftime("%b %d, %Y %H:%M UTC")
            except (ValueError, TypeError):
                timestamp = ""
                date_published = str(date_published)
        else:
            timestamp = ""
            date_published = "Unknown Date"

        # Location
        country_code = raw_item.get("country", "")
        country = COUNTRY_NAMES.get(country_code, country_code) if country_code else "Unknown"
        state = raw_item.get("state", "")
        city = raw_item.get("city", "")

        location_parts = []
        if city:
            location_parts.append(city)
        if state and country == "United States":
            location_parts.append(state)
        if country and country != "Unknown":
            location_parts.append(country)
        location_display = ", ".join(location_parts) if location_parts else "Location Unknown"

        url = raw_item.get("url", f"https://www.dvidshub.net/{content_type}/{item_id}")
        thumbnail_url = raw_item.get("thumbnail", raw_item.get("thumbnail_url", ""))

        keywords = raw_item.get("keywords", "").split(",") if raw_item.get("keywords") else []
        keywords = [k.strip() for k in keywords if k.strip()]

        credit = raw_item.get("credit", raw_item.get("author", ""))
        duration = raw_item.get("duration", "")
        aspect_ratio = raw_item.get("aspect_ratio", "")

        return DVIDSItem(
            id=item_id, title=title, description=description,
            type=content_type, branch=branch, unit_name=unit_name,
            date_published=date_published, timestamp=timestamp,
            country=country, state=state, city=city,
            location_display=location_display, url=url,
            thumbnail_url=thumbnail_url, keywords=keywords,
            credit=credit, duration=duration, aspect_ratio=aspect_ratio,
        )

    except Exception as e:
        print(f"Error parsing item: {e}")
        return None


def fetch_daily_content() -> List[DVIDSItem]:
    """Fetch all Navy-related content from the configured time period."""
    date = datetime.utcnow()
    to_date = date.strftime("%Y-%m-%dT%H:%M:%SZ")
    from_date = (date - timedelta(hours=LOOKBACK_HOURS)).strftime("%Y-%m-%dT%H:%M:%SZ")

    print("="*60)
    print("DVIDS DAILY DIGEST - FETCHING CONTENT")
    print(f"Time Range: {from_date} to {to_date}")
    print("="*60)

    all_items = []
    seen_ids = set()

    for branch in BRANCHES:
        for content_type in CONTENT_TYPES:
            print(f"Fetching {branch} {content_type}...", end=" ")

            results = search_dvids(
                content_type=content_type,
                branch=branch,
                from_date=from_date,
                to_date=to_date,
                max_results=MAX_RESULTS_PER_QUERY,
            )

            count = 0
            for raw_item in results:
                item = parse_dvids_item(raw_item)
                if item and item.id not in seen_ids:
                    seen_ids.add(item.id)
                    all_items.append(item)
                    count += 1

            print(f"{count} items")
            time.sleep(0.5)

    all_items.sort(key=lambda x: x.timestamp, reverse=True)

    print("="*60)
    print(f"Total unique items: {len(all_items)}")
    print("="*60)

    return all_items


def create_daily_digest(items: List[DVIDSItem]) -> DailyDigest:
    """Create a DailyDigest summary."""
    date_str = datetime.utcnow().strftime("%Y-%m-%d")

    by_country = defaultdict(int)
    by_type = defaultdict(int)
    by_branch = defaultdict(int)

    for item in items:
        by_country[item.country] += 1
        by_type[item.type] += 1
        by_branch[item.branch] += 1

    return DailyDigest(
        date=date_str,
        items=items,
        total_count=len(items),
        by_country=dict(by_country),
        by_type=dict(by_type),
        by_branch=dict(by_branch),
    )


print("Data processing functions loaded!")

In [ ]:
# ============================================================================
# RUN THE SCRAPER
# ============================================================================

if DVIDS_API_KEY == "YOUR_API_KEY_HERE":
    print("ERROR: Please set your DVIDS API key!")
    print("Get a free key at: https://api.dvidshub.net/docs")
    print("Then update DVIDS_API_KEY in the configuration cell.")
else:
    # Fetch content
    items = fetch_daily_content()

    # Create digest
    digest = create_daily_digest(items)

    # Display summary
    print("\n" + "="*60)
    print("SUMMARY")
    print("="*60)
    print(f"Total items: {digest.total_count}")
    print(f"News: {digest.by_type.get('news', 0)}")
    print(f"Images: {digest.by_type.get('image', 0)}")
    print(f"Videos: {digest.by_type.get('video', 0)}")
    print("\nTop Countries:")
    for country, count in sorted(digest.by_country.items(), key=lambda x: x[1], reverse=True)[:5]:
        print(f"  {country}: {count}")

In [ ]:
# ============================================================================
# GENERATE AND DOWNLOAD HTML
# ============================================================================

# Import HTML generator (simplified version for Colab)
def generate_dvids_html(digest: DailyDigest) -> str:
    """Generate the DVIDS News HTML page."""

    items_json = json.dumps([asdict(item) for item in digest.items])
    by_country_json = json.dumps(digest.by_country)
    by_type_json = json.dumps(digest.by_type)
    by_branch_json = json.dumps(digest.by_branch)

    timestamp = datetime.utcnow().strftime("%Y-%m-%d %H:%M UTC")

    news_count = digest.by_type.get("news", 0)
    image_count = digest.by_type.get("image", 0)
    video_count = digest.by_type.get("video", 0)

    # Full HTML template (abbreviated here - see dvids_scraper.py for complete version)
    html = f'''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>DVIDS DAILY DIGEST - U.S. Navy News</title>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap" rel="stylesheet">
    <style>
        * {{ margin: 0; padding: 0; box-sizing: border-box; }}
        body {{ background: #0a0a0f; color: #e0e0e0; font-family: 'Inter', sans-serif; min-height: 100vh; }}
        .header {{ background: linear-gradient(180deg, #111118 0%, #0a0a0f 100%); border-bottom: 1px solid #1e1e2e; padding: 16px 24px; position: sticky; top: 0; z-index: 100; }}
        .header-content {{ max-width: 1400px; margin: 0 auto; display: flex; justify-content: space-between; align-items: center; flex-wrap: wrap; gap: 16px; }}
        .logo {{ font-size: 18px; font-weight: 700; color: #00ff88; }}
        .logo-sub {{ font-size: 10px; color: #666; letter-spacing: 2px; margin-top: 2px; text-transform: uppercase; }}
        .stats-bar {{ display: flex; gap: 24px; flex-wrap: wrap; }}
        .stat {{ text-align: center; padding: 8px 16px; background: rgba(255,255,255,0.02); border-radius: 8px; }}
        .stat-value {{ font-size: 24px; font-weight: 700; }}
        .stat-value.total {{ color: #00ffff; }}
        .stat-value.news {{ color: #ff6b6b; }}
        .stat-value.image {{ color: #4ecdc4; }}
        .stat-value.video {{ color: #ffd93d; }}
        .stat-label {{ font-size: 9px; font-weight: 600; color: #555; text-transform: uppercase; margin-top: 2px; }}
        .timestamp {{ font-size: 11px; color: #444; }}
        .timestamp span {{ color: #00ff88; }}
        .main-container {{ max-width: 1400px; margin: 0 auto; padding: 20px; display: grid; grid-template-columns: 280px 1fr; gap: 20px; }}
        .sidebar {{ background: #0d0d14; border: 1px solid #1e1e2e; border-radius: 12px; padding: 16px; height: fit-content; position: sticky; top: 100px; }}
        .sidebar-title {{ font-size: 11px; font-weight: 600; color: #00ff88; letter-spacing: 2px; text-transform: uppercase; margin-bottom: 16px; }}
        .filter-section {{ margin-bottom: 20px; }}
        .filter-label {{ font-size: 10px; font-weight: 600; color: #666; text-transform: uppercase; margin-bottom: 8px; }}
        .filter-group {{ display: flex; flex-direction: column; gap: 6px; }}
        .filter-btn {{ font-family: 'Inter', sans-serif; font-size: 11px; padding: 8px 12px; background: transparent; border: 1px solid #2a2a3a; color: #888; cursor: pointer; border-radius: 6px; text-align: left; display: flex; justify-content: space-between; }}
        .filter-btn:hover {{ border-color: #444; color: #ccc; }}
        .filter-btn.active {{ background: rgba(0, 255, 136, 0.1); border-color: #00ff88; color: #00ff88; }}
        .filter-count {{ font-size: 10px; color: #555; }}
        .filter-btn.active .filter-count {{ color: #00ff88; }}
        .search-box {{ margin-bottom: 16px; }}
        .search-input {{ width: 100%; padding: 10px 12px; background: rgba(255,255,255,0.02); border: 1px solid #2a2a3a; border-radius: 8px; color: #e0e0e0; font-family: 'Inter', sans-serif; font-size: 12px; }}
        .search-input:focus {{ outline: none; border-color: #00ff88; }}
        .search-input::placeholder {{ color: #555; }}
        .content-area {{ min-height: 80vh; }}
        .content-header {{ display: flex; justify-content: space-between; align-items: center; margin-bottom: 16px; }}
        .results-count {{ font-size: 12px; color: #888; }}
        .view-toggle {{ display: flex; gap: 4px; }}
        .view-btn {{ font-family: 'Inter', sans-serif; font-size: 11px; padding: 6px 12px; background: transparent; border: 1px solid #2a2a3a; color: #666; cursor: pointer; border-radius: 6px; }}
        .view-btn.active {{ background: rgba(0, 255, 136, 0.1); border-color: #00ff88; color: #00ff88; }}
        .location-group {{ margin-bottom: 24px; }}
        .location-header {{ display: flex; align-items: center; gap: 10px; padding: 12px 16px; background: rgba(0, 255, 136, 0.05); border-radius: 10px; margin-bottom: 12px; cursor: pointer; }}
        .location-header:hover {{ background: rgba(0, 255, 136, 0.08); }}
        .region-badge {{ font-size: 9px; font-weight: 600; color: #00ffff; background: rgba(0, 255, 255, 0.1); padding: 3px 8px; border-radius: 4px; text-transform: uppercase; }}
        .location-name {{ flex: 1; font-size: 14px; font-weight: 600; color: #00ff88; }}
        .location-count {{ font-size: 11px; font-weight: 600; color: #888; background: rgba(255,255,255,0.05); padding: 4px 10px; border-radius: 12px; }}
        .items-grid {{ display: grid; grid-template-columns: repeat(auto-fill, minmax(320px, 1fr)); gap: 16px; padding-left: 12px; }}
        .item-card {{ background: rgba(255, 255, 255, 0.02); border: 1px solid #1e1e2e; border-radius: 12px; overflow: hidden; cursor: pointer; transition: all 0.2s; }}
        .item-card:hover {{ border-color: #333; background: rgba(255, 255, 255, 0.04); transform: translateY(-2px); }}
        .item-card.news {{ border-left: 4px solid #ff6b6b; }}
        .item-card.image {{ border-left: 4px solid #4ecdc4; }}
        .item-card.video {{ border-left: 4px solid #ffd93d; }}
        .item-thumbnail {{ width: 100%; height: 160px; object-fit: cover; background: #1a1a2e; }}
        .item-content {{ padding: 14px 16px; }}
        .item-meta {{ display: flex; justify-content: space-between; align-items: center; margin-bottom: 8px; }}
        .item-type {{ font-size: 9px; font-weight: 600; padding: 4px 8px; border-radius: 4px; text-transform: uppercase; }}
        .item-type.news {{ background: rgba(255, 107, 107, 0.15); color: #ff6b6b; }}
        .item-type.image {{ background: rgba(78, 205, 196, 0.15); color: #4ecdc4; }}
        .item-type.video {{ background: rgba(255, 217, 61, 0.15); color: #ffd93d; }}
        .item-branch {{ font-size: 10px; color: #666; }}
        .item-title {{ font-size: 14px; font-weight: 600; color: #fff; line-height: 1.4; margin-bottom: 8px; display: -webkit-box; -webkit-line-clamp: 2; -webkit-box-orient: vertical; overflow: hidden; }}
        .item-description {{ font-size: 12px; color: #888; line-height: 1.5; display: -webkit-box; -webkit-line-clamp: 3; -webkit-box-orient: vertical; overflow: hidden; margin-bottom: 10px; }}
        .item-footer {{ display: flex; justify-content: space-between; align-items: center; }}
        .item-unit {{ font-size: 10px; color: #555; max-width: 200px; white-space: nowrap; overflow: hidden; text-overflow: ellipsis; }}
        .item-date {{ font-size: 10px; color: #444; }}
        .modal-overlay {{ position: fixed; top: 0; left: 0; right: 0; bottom: 0; background: rgba(0, 0, 0, 0.9); display: flex; align-items: center; justify-content: center; z-index: 1000; opacity: 0; visibility: hidden; transition: all 0.3s ease; padding: 20px; }}
        .modal-overlay.visible {{ opacity: 1; visibility: visible; }}
        .modal {{ background: #111118; border: 1px solid #2a2a3a; border-radius: 16px; width: 100%; max-width: 800px; max-height: 90vh; overflow: hidden; transform: scale(0.9); transition: transform 0.3s ease; }}
        .modal-overlay.visible .modal {{ transform: scale(1); }}
        .modal-header {{ padding: 20px 24px; border-bottom: 1px solid #1e1e2e; display: flex; justify-content: space-between; align-items: flex-start; }}
        .modal-title {{ font-size: 18px; font-weight: 700; color: #fff; line-height: 1.4; flex: 1; padding-right: 16px; }}
        .modal-close {{ cursor: pointer; color: #555; font-size: 28px; width: 40px; height: 40px; display: flex; align-items: center; justify-content: center; border-radius: 8px; }}
        .modal-close:hover {{ color: #ff6b6b; background: rgba(255, 107, 107, 0.1); }}
        .modal-body {{ padding: 24px; overflow-y: auto; max-height: calc(90vh - 180px); }}
        .modal-image {{ width: 100%; max-height: 400px; object-fit: contain; background: #0a0a0f; border-radius: 8px; margin-bottom: 20px; }}
        .modal-meta {{ display: flex; flex-wrap: wrap; gap: 12px; margin-bottom: 20px; }}
        .modal-tag {{ font-size: 11px; padding: 6px 12px; border-radius: 6px; background: rgba(255,255,255,0.05); color: #888; }}
        .modal-tag.type {{ color: #00ffff; background: rgba(0, 255, 255, 0.1); }}
        .modal-tag.branch {{ color: #ff6b6b; background: rgba(255, 107, 107, 0.1); }}
        .modal-tag.location {{ color: #00ff88; background: rgba(0, 255, 136, 0.1); }}
        .modal-description {{ font-size: 14px; color: #ccc; line-height: 1.7; margin-bottom: 20px; }}
        .modal-info {{ display: grid; grid-template-columns: repeat(auto-fill, minmax(200px, 1fr)); gap: 12px; }}
        .modal-info-item {{ background: rgba(255,255,255,0.02); padding: 12px; border-radius: 8px; }}
        .modal-info-label {{ font-size: 10px; font-weight: 600; color: #555; text-transform: uppercase; margin-bottom: 4px; }}
        .modal-info-value {{ font-size: 13px; color: #aaa; }}
        .modal-footer {{ padding: 16px 24px; border-top: 1px solid #1e1e2e; display: flex; justify-content: flex-end; }}
        .modal-btn {{ font-family: 'Inter', sans-serif; font-size: 12px; font-weight: 600; padding: 10px 20px; border: none; border-radius: 8px; cursor: pointer; background: linear-gradient(135deg, #00ff88 0%, #00cc6a 100%); color: #000; }}
        .modal-btn:hover {{ transform: translateY(-2px); box-shadow: 0 4px 12px rgba(0, 255, 136, 0.3); }}
        .empty-state {{ text-align: center; padding: 60px 20px; color: #555; }}
        .attribution {{ text-align: center; padding: 24px; font-size: 10px; color: #444; }}
        @media (max-width: 900px) {{ .main-container {{ grid-template-columns: 1fr; }} .sidebar {{ position: static; }} }}
        @media (max-width: 600px) {{ .items-grid {{ grid-template-columns: 1fr; }} }}
    </style>
</head>
<body>
    <header class="header">
        <div class="header-content">
            <div><div class="logo">DVIDS DAILY DIGEST</div><div class="logo-sub">U.S. Navy &amp; Marine Corps News</div></div>
            <div class="stats-bar">
                <div class="stat"><div class="stat-value total">{digest.total_count}</div><div class="stat-label">Total Items</div></div>
                <div class="stat"><div class="stat-value news">{news_count}</div><div class="stat-label">News</div></div>
                <div class="stat"><div class="stat-value image">{image_count}</div><div class="stat-label">Images</div></div>
                <div class="stat"><div class="stat-value video">{video_count}</div><div class="stat-label">Videos</div></div>
            </div>
            <div class="timestamp">Last Update: <span>{timestamp}</span></div>
        </div>
    </header>
    <div class="main-container">
        <aside class="sidebar">
            <div class="sidebar-title">Filters</div>
            <div class="search-box"><input type="text" class="search-input" id="searchInput" placeholder="Search titles, units..." oninput="filterItems()"></div>
            <div class="filter-section"><div class="filter-label">Content Type</div><div class="filter-group" id="typeFilters"></div></div>
            <div class="filter-section"><div class="filter-label">Branch</div><div class="filter-group" id="branchFilters"></div></div>
            <div class="filter-section"><div class="filter-label">Country</div><div class="filter-group" id="countryFilters"></div></div>
        </aside>
        <main class="content-area">
            <div class="content-header">
                <div class="results-count" id="resultsCount">Showing {digest.total_count} items</div>
                <div class="view-toggle">
                    <button class="view-btn active" data-view="grid" onclick="setView('grid')">Grid</button>
                    <button class="view-btn" data-view="list" onclick="setView('list')">List</button>
                </div>
            </div>
            <div id="contentContainer"></div>
        </main>
    </div>
    <div class="attribution">Created by @ianellisjones and IEJ Media | Data from DVIDS (dvidshub.net)</div>
    <div class="modal-overlay" id="detailModal" onclick="closeModal(event)">
        <div class="modal" onclick="event.stopPropagation()">
            <div class="modal-header"><div class="modal-title" id="modalTitle">-</div><span class="modal-close" onclick="closeModal()">&times;</span></div>
            <div class="modal-body" id="modalBody"></div>
            <div class="modal-footer"><a id="modalLink" href="#" target="_blank"><button class="modal-btn">View on DVIDS</button></a></div>
        </div>
    </div>
    <script>
        const allItems = {items_json};
        const byCountry = {by_country_json};
        const byType = {by_type_json};
        const byBranch = {by_branch_json};
        const regionMap = {json.dumps(REGION_MAP)};
        let currentFilters = {{ type: 'all', branch: 'all', country: 'all', search: '' }};
        let currentView = 'grid';

        function initFilters() {{
            const total = allItems.length;
            document.getElementById('typeFilters').innerHTML = '<button class="filter-btn active" data-type="all" onclick="setTypeFilter(\'all\')">All <span class="filter-count">' + total + '</span></button>' + Object.entries(byType).map(([t, c]) => '<button class="filter-btn" data-type="' + t + '" onclick="setTypeFilter(\'' + t + '\')">' + t.charAt(0).toUpperCase() + t.slice(1) + ' <span class="filter-count">' + c + '</span></button>').join('');
            document.getElementById('branchFilters').innerHTML = '<button class="filter-btn active" data-branch="all" onclick="setBranchFilter(\'all\')">All <span class="filter-count">' + total + '</span></button>' + Object.entries(byBranch).map(([b, c]) => '<button class="filter-btn" data-branch="' + b + '" onclick="setBranchFilter(\'' + b + '\')">' + b + ' <span class="filter-count">' + c + '</span></button>').join('');
            document.getElementById('countryFilters').innerHTML = '<button class="filter-btn active" data-country="all" onclick="setCountryFilter(\'all\')">All <span class="filter-count">' + total + '</span></button>' + Object.entries(byCountry).sort((a,b) => b[1]-a[1]).map(([co, c]) => '<button class="filter-btn" data-country="' + co + '" onclick="setCountryFilter(\'' + co + '\')">' + co + ' <span class="filter-count">' + c + '</span></button>').join('');
        }}

        function setTypeFilter(t) {{ currentFilters.type = t; document.querySelectorAll('#typeFilters .filter-btn').forEach(b => b.classList.toggle('active', b.dataset.type === t)); renderItems(); }}
        function setBranchFilter(b) {{ currentFilters.branch = b; document.querySelectorAll('#branchFilters .filter-btn').forEach(btn => btn.classList.toggle('active', btn.dataset.branch === b)); renderItems(); }}
        function setCountryFilter(c) {{ currentFilters.country = c; document.querySelectorAll('#countryFilters .filter-btn').forEach(btn => btn.classList.toggle('active', btn.dataset.country === c)); renderItems(); }}
        function filterItems() {{ currentFilters.search = document.getElementById('searchInput').value.toLowerCase(); renderItems(); }}
        function setView(v) {{ currentView = v; document.querySelectorAll('.view-btn').forEach(btn => btn.classList.toggle('active', btn.dataset.view === v)); renderItems(); }}

        function getFilteredItems() {{
            return allItems.filter(item => {{
                if (currentFilters.type !== 'all' && item.type !== currentFilters.type) return false;
                if (currentFilters.branch !== 'all' && item.branch !== currentFilters.branch) return false;
                if (currentFilters.country !== 'all' && item.country !== currentFilters.country) return false;
                if (currentFilters.search && !item.title.toLowerCase().includes(currentFilters.search) && !item.unit_name.toLowerCase().includes(currentFilters.search)) return false;
                return true;
            }});
        }}

        function renderItems() {{
            const filtered = getFilteredItems();
            document.getElementById('resultsCount').textContent = 'Showing ' + filtered.length + ' items';
            if (filtered.length === 0) {{ document.getElementById('contentContainer').innerHTML = '<div class="empty-state">No items found</div>'; return; }}
            const grouped = {{}};
            filtered.forEach(item => {{ if (!grouped[item.country]) grouped[item.country] = []; grouped[item.country].push(item); }});
            let html = '';
            Object.keys(grouped).sort((a, b) => grouped[b].length - grouped[a].length).forEach(country => {{
                const items = grouped[country];
                const region = regionMap[country] || 'Other';
                html += '<div class="location-group"><div class="location-header"><span class="region-badge">' + region + '</span><span class="location-name">' + country + '</span><span class="location-count">' + items.length + '</span></div>';
                html += '<div class="items-grid">';
                items.forEach(item => {{
                    html += '<div class="item-card ' + item.type + '" onclick="showDetail(\'' + item.id + '\')">';
                    if (item.thumbnail_url) html += '<img class="item-thumbnail" src="' + item.thumbnail_url + '" loading="lazy" onerror="this.style.display=\'none\'">';
                    html += '<div class="item-content"><div class="item-meta"><span class="item-type ' + item.type + '">' + item.type + '</span><span class="item-branch">' + item.branch + '</span></div>';
                    html += '<div class="item-title">' + item.title + '</div>';
                    html += '<div class="item-description">' + (item.description || '') + '</div>';
                    html += '<div class="item-footer"><span class="item-unit">' + item.unit_name + '</span><span class="item-date">' + item.date_published + '</span></div></div></div>';
                }});
                html += '</div></div>';
            }});
            document.getElementById('contentContainer').innerHTML = html;
        }}

        function showDetail(id) {{
            const item = allItems.find(i => i.id === id);
            if (!item) return;
            document.getElementById('modalTitle').textContent = item.title;
            document.getElementById('modalLink').href = item.url;
            let html = '';
            if (item.thumbnail_url) html += '<img class="modal-image" src="' + item.thumbnail_url + '">';
            html += '<div class="modal-meta"><span class="modal-tag type">' + item.type + '</span><span class="modal-tag branch">' + item.branch + '</span><span class="modal-tag location">' + item.location_display + '</span></div>';
            if (item.description) html += '<div class="modal-description">' + item.description + '</div>';
            html += '<div class="modal-info"><div class="modal-info-item"><div class="modal-info-label">Unit</div><div class="modal-info-value">' + item.unit_name + '</div></div>';
            html += '<div class="modal-info-item"><div class="modal-info-label">Published</div><div class="modal-info-value">' + item.date_published + '</div></div></div>';
            document.getElementById('modalBody').innerHTML = html;
            document.getElementById('detailModal').classList.add('visible');
            document.body.style.overflow = 'hidden';
        }}

        function closeModal(e) {{ if (e && e.target !== e.currentTarget) return; document.getElementById('detailModal').classList.remove('visible'); document.body.style.overflow = ''; }}
        document.addEventListener('keydown', e => {{ if (e.key === 'Escape') closeModal(); }});
        initFilters();
        renderItems();
    </script>
</body>
</html>'''

    return html


# Generate HTML
if 'digest' in dir() and digest.total_count > 0:
    html_content = generate_dvids_html(digest)

    # Save to file
    with open('dvids_daily_digest.html', 'w', encoding='utf-8') as f:
        f.write(html_content)

    # Also save JSON
    json_data = {
        "date": digest.date,
        "timestamp": datetime.utcnow().isoformat(),
        "total_count": digest.total_count,
        "by_country": digest.by_country,
        "by_type": digest.by_type,
        "by_branch": digest.by_branch,
        "items": [asdict(item) for item in digest.items]
    }
    with open('dvids_data.json', 'w', encoding='utf-8') as f:
        json.dump(json_data, f, indent=2)

    print("\nFiles generated successfully!")
    print("- dvids_daily_digest.html")
    print("- dvids_data.json")

    # Download files
    files.download('dvids_daily_digest.html')
    files.download('dvids_data.json')
else:
    print("No data to generate. Run the scraper cell first.")

---

## Notes

- **API Key**: Get your free key at [https://api.dvidshub.net/docs](https://api.dvidshub.net/docs)
- **Rate Limits**: The scraper includes automatic retry logic for rate limiting
- **Output**: Downloads both HTML (web page) and JSON (raw data) files
- **GitHub Pages**: Upload the HTML file to your repo to publish via GitHub Pages

For questions or issues, visit: [github.com/ianellisjones/usn](https://github.com/ianellisjones/usn)